In [6]:
import string 

import pandas as pd
import numpy as np
import re
from airflow import DAG
import pymongo

In [2]:
data = pd.read_csv('tiktok_google_play_reviews.csv')
data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHRz-11c0apHLSKHHp52FxUXsQS9Z88wP3sWc5...,MR LOL GAMER,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,5,0,23.8.4,2022-04-05 23:18:30,NaN,NaN
1,gp:AOqpTOF6mFDEkIypmyT3shDLjPHg8zB3kdns2W36ahp...,Dino Kljako,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome app! Too many people on it where it's ...,5,0,NaN,2022-04-05 23:18:21,NaN,NaN
2,gp:AOqpTOGtqU4sb8vuVo3-eB7kIXWoBn-0YCUZ1SnPRKS...,Olivia Harding,https://play-lh.googleusercontent.com/a/AATXAJ...,Not bad,5,0,23.9.5,2022-04-05 23:17:34,NaN,NaN
3,gp:AOqpTOFHDm-Qa5R6jCpOGTFT2qr1_PKbCTbBNPahCEn...,Keli We,https://play-lh.googleusercontent.com/a-/AOh14...,It is good,2,0,22.2.5,2022-04-05 23:17:04,NaN,NaN
4,gp:AOqpTOFB6Ndao8IHRpOJRmbSknwMGxHcwYzux93YyXI...,Mavis Kotoka,https://play-lh.googleusercontent.com/a/AATXAJ...,Very interesting app,5,0,22.1.5,2022-04-05 23:17:04,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307057 entries, 0 to 307056
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              307057 non-null  object
 1   userName              307057 non-null  object
 2   userImage             307057 non-null  object
 3   content               307053 non-null  object
 4   score                 307057 non-null  int64 
 5   thumbsUpCount         307057 non-null  int64 
 6   reviewCreatedVersion  217841 non-null  object
 7   at                    307057 non-null  object
 8   replyContent          119 non-null     object
 9   repliedAt             119 non-null     object
dtypes: int64(2), object(8)
memory usage: 23.4+ MB


In [4]:
# data_notna = data[data.replyContent.notna()]
# data_notna.replyContent

In [5]:
# data_notna.replyContent[297821]

In [6]:
# data_notna.reset_index(inplace=True)

In [7]:
# Delete all rows where all values are NaN
for i in range(data.shape[0]):
    if all(data.loc[i].isna()):
        data.drop(i, axis=0, inplace=True)

In [8]:
data.fillna("-", inplace=True)
data.sort_values(by="at", inplace=True)

data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
307056,gp:AOqpTOF_2SyR6RmxASgEiW-4Vw9QRwF4sybqcSdnTlq...,Sumaiya Islam,https://play-lh.googleusercontent.com/a/AATXAJ...,plz sapport my id,5,0,22.7.4,2022-01-17 10:41:51,-,-
307055,gp:AOqpTOFoaOhvxddyVQPQJHpv9f4kgn2u2kkniAWQTRM...,Asika Emeka,https://play-lh.googleusercontent.com/a/AATXAJ...,Nice app,5,0,-,2022-01-17 10:42:27,-,-
307054,gp:AOqpTOEuzjdo0NhbqpSBB18hhKP8X6FNPDOWVsan_TA...,ntokozo zondi,https://play-lh.googleusercontent.com/a/AATXAJ...,The videos are amazing...such an amazing app,5,0,22.8.4,2022-01-17 10:43:06,-,-
307053,gp:AOqpTOHnBTSFCqxFYSsNrg4vztYn-Z0Gg3aCIGLliRK...,Shalina Sayed,https://play-lh.googleusercontent.com/a/AATXAJ...,I LOVE TIKTOK,5,0,22.8.4,2022-01-17 10:44:31,-,-
307052,gp:AOqpTOEX2qzcEhnamYxu8Jrnu4Ia0UtuHkajx6I1Y_j...,Nze Alpeace,https://play-lh.googleusercontent.com/a/AATXAJ...,"Best app ever let's do it ,very awesome let ev...",5,0,22.6.5,2022-01-17 10:44:38,-,-


In [9]:
for i in range(0, len(data)):
    # Remove special symbols
    data.replyContent[i] = re.sub('\n', ' ', str(data['replyContent'][i]))
            # Remove all symbols except letters
    data.replyContent[i] = re.sub(f'[^a-zA-Z{string.punctuation}]', ' ', str(data['replyContent'][i]))
    # Replacing all gaps with spaces 
    data.replyContent[i] = re.sub(r'\s+', ' ', data.replyContent[i])                    

/home/vermichel/anaconda3/envs/de/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/vermichel/anaconda3/envs/de/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/vermichel/anaconda3/envs/de/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [10]:
data

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
307056,gp:AOqpTOF_2SyR6RmxASgEiW-4Vw9QRwF4sybqcSdnTlq...,Sumaiya Islam,https://play-lh.googleusercontent.com/a/AATXAJ...,plz sapport my id,5,0,22.7.4,2022-01-17 10:41:51,-,-
307055,gp:AOqpTOFoaOhvxddyVQPQJHpv9f4kgn2u2kkniAWQTRM...,Asika Emeka,https://play-lh.googleusercontent.com/a/AATXAJ...,Nice app,5,0,-,2022-01-17 10:42:27,-,-
307054,gp:AOqpTOEuzjdo0NhbqpSBB18hhKP8X6FNPDOWVsan_TA...,ntokozo zondi,https://play-lh.googleusercontent.com/a/AATXAJ...,The videos are amazing...such an amazing app,5,0,22.8.4,2022-01-17 10:43:06,-,-
307053,gp:AOqpTOHnBTSFCqxFYSsNrg4vztYn-Z0Gg3aCIGLliRK...,Shalina Sayed,https://play-lh.googleusercontent.com/a/AATXAJ...,I LOVE TIKTOK,5,0,22.8.4,2022-01-17 10:44:31,-,-
307052,gp:AOqpTOEX2qzcEhnamYxu8Jrnu4Ia0UtuHkajx6I1Y_j...,Nze Alpeace,https://play-lh.googleusercontent.com/a/AATXAJ...,"Best app ever let's do it ,very awesome let ev...",5,0,22.6.5,2022-01-17 10:44:38,-,-
...,...,...,...,...,...,...,...,...,...,...
4,gp:AOqpTOFB6Ndao8IHRpOJRmbSknwMGxHcwYzux93YyXI...,Mavis Kotoka,https://play-lh.googleusercontent.com/a/AATXAJ...,Very interesting app,5,0,22.1.5,2022-04-05 23:17:04,-,-
3,gp:AOqpTOFHDm-Qa5R6jCpOGTFT2qr1_PKbCTbBNPahCEn...,Keli We,https://play-lh.googleusercontent.com/a-/AOh14...,It is good,2,0,22.2.5,2022-04-05 23:17:04,-,-
2,gp:AOqpTOGtqU4sb8vuVo3-eB7kIXWoBn-0YCUZ1SnPRKS...,Olivia Harding,https://play-lh.googleusercontent.com/a/AATXAJ...,Not bad,5,0,23.9.5,2022-04-05 23:17:34,-,-
1,gp:AOqpTOF6mFDEkIypmyT3shDLjPHg8zB3kdns2W36ahp...,Dino Kljako,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome app! Too many people on it where it's ...,5,0,-,2022-04-05 23:18:21,-,-


In [12]:
data['score'].astype(int)
data['thumbsUpCount'].astype(int)

307056    0
307055    0
307054    0
307053    0
307052    0
         ..
4         0
3         0
2         0
1         0
0         0
Name: thumbsUpCount, Length: 307057, dtype: int64

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307057 entries, 307056 to 0
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              307057 non-null  object
 1   userName              307057 non-null  object
 2   userImage             307057 non-null  object
 3   content               307057 non-null  object
 4   score                 307057 non-null  int64 
 5   thumbsUpCount         307057 non-null  int64 
 6   reviewCreatedVersion  307057 non-null  object
 7   at                    307057 non-null  object
 8   replyContent          307057 non-null  object
 9   repliedAt             307057 non-null  object
dtypes: int64(2), object(8)
memory usage: 33.8+ MB


In [20]:
data.score.value_counts()

5    232507
1     37080
4     17985
3     11652
2      7833
Name: score, dtype: int64

In [18]:
data.to_csv('mongo_data.csv', index=False)

In [2]:
best_comments_pipeline = [
    {
        '$group': {
            '_id': '$_id', 
            'total_score': {
                '$sum': {
                    '$toInt': '$score'
                }
            }
        }
    }, {
        '$sort': {
            'total_score': -1
        }
    }, {
        '$limit': 5
    }
]

In [3]:
short_content_pipeline = [
    {
        '$project': {
            '_id': '$_id', 
            'reviewId': '$reviewId', 
            'userName': '$userName', 
            'userImage': '$userImage', 
            'content': '$content', 
            'score': '$score', 
            'thumbsUpCount': '$thumbsUpCount', 
            'reviewCreatedVersion': '$reviewCreatedVersion', 
            'at': '$at', 
            'replyContent': '$replyContent', 
            'length': {
                '$strLenCP': '$content'
            }
        }
    }, {
        '$match': {
            'length': {
                '$lt': 5
            }
        }
    }
]

In [4]:
day_rating_pipeline = [
    {
        '$project': {
            'score': {
                '$toInt': '$score'
            }, 
            'at': {
                '$dateFromString': {
                    'dateString': '$at'
                }
            }
        }
    }, {
        '$group': {
            '_id': {
                '$dayOfYear': '$at'
            }, 
            'average': {
                '$avg': '$score'
            }
        }
    }
]

In [12]:
def get_data_from_mongo(collection: MongoClient, pipeline_func: list) -> dict:
    """Getting pre-aggregated data from MongoDB"""
    
    return list(collection.aggregate(pipeline_func()))

In [4]:
def main():
    client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
    db = client['local']   
    coll = db['best_comments']
    best_comments = coll.insert(aggregate(best_comments_pipeline))
    coll = db['short_content']
    short_content = coll.insert(aggregate(short_content_pipeline))
    coll = db['day_rating']
    day_rating = coll.insert(aggregate(day_rating_pipeline))
    coll.insert_many(data_dict)

In [7]:
with DAG(
    dag_id = 'daily_report',
    dag_args = {
        'retries': 1,
        'retry_delay' = timedelta(minutes=20)
    },
    'catchup': False,
    'schedule_interval' = '0 0 * * *',
    start_time = datetime(27, 8, 2022)
)
as dag:
    t1 = PythonOperator(
        task_id = 'daily',
        python_callable = main,
        depends_on_past=False,
        retries=0
    )

SyntaxError: invalid syntax (1610702954.py, line 5)